In [100]:
import json, os, re, requests, sys, time
import pandas as pd
import numpy as np
import logging
import logging.handlers

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from datetime import date, datetime
from pykrx import stock
from webob.compat import urlparse
import lxml
url_dict = {'휘발유': 'https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=%s',
            '고급휘발유':'https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_HGSL&page=%s',
            '경유':'https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_LO&page=%s',
            '두바이유':'https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=OIL_DU&fdtc=2&page=%s',
            '브렌트유':'https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=OIL_BRT&fdtc=2&page=%s',
            'WTI(서부텍사스유)':'https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=OIL_CL&fdtc=2&page=%s'
            }


In [4]:
url_dict.keys()

dict_keys(['휘발유', '고급휘발유', '경유', '두바이유', '브렌트유', 'WTI(서부텍사스유)'])

In [238]:
url_dict['휘발유']

'https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=%s'

In [ ]:
df = pd.DataFrame()

for key in url_dict.keys():
    
    values = []
    
    page = 1
    page_text_len = 100

    print(key+ '자료 수집 시작 ####')
    
    while page_text_len > 10:
        r_url = url_dict[key] % page
        print('r_url = ',r_url)
        r = requests.get(r_url)
        soup = bs(r.text,'lxml')
        tbody = soup.find('tbody')
        tr = tbody.find_all('tr')
        
        page_text_len = len(tbody.text.strip())
        # print(page_text_len)

        for t in tr:
            x = t.text.strip().replace('+','').replace('\t','').split('\n')
            x = ' '.join(x).split()
            values.append(x)
            
        page += 1

    # 전일대비값 : text->flaot 변환 및 등락율의 부호에 따라 부호 수정
    # 종가 : text->flaot 변환
    
    for i in values:
        i[1] = float(i[1].replace(',',''))
        i[2] = float(i[2])
        if i[3][0] == "-":
            i[2] = i[2] * (-1)
        else: pass
        
        
    # print(values)

    date = []
    key_values = []
    for i in range(len(values)):
        x = values[i][0] # 일자
        y = values[i][1] # 종가
        date.append(x)
        key_values.append(y)

    df_temp = pd.DataFrame(key_values, index=date, columns=[key])
    df = pd.merge(df, df_temp, how='outer', left_index=True, right_index=True)
    
    print(key + '자료 수집 완료 ####')
    time.sleep(10)
            

휘발유자료 수집 시작 ####
r_url =  https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=1
322
r_url =  https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=2
324
r_url =  https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=3
330
r_url =  https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=4
341
r_url =  https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=5
336
r_url =  https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=6
334
r_url =  https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=7
334
r_url =  https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=8
334
r_url =  https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=9
334
r_url =  https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OI

In [406]:
import time
import pickle

df_1 = df.copy()
df_1 = df_1.sort_index(ascending=False)
# df_1.sort_index(ascending=False).reset_index(drop=False).rename(columns = {'index':'일자'})

%time df_1.to_pickle('oildata.pickle')

In [412]:
df_1

,휘발유,고급휘발유,경유,두바이유,브렌트유,WTI(서부텍사스유)
2022.04.03,1993.98,2202.69,1914.65,NaN,NaN,NaN
2022.04.02,1994.51,2204.60,1915.19,NaN,NaN,NaN
2022.04.01,1996.48,2211.32,1917.02,101.85,104.39,99.27
2022.03.31,1998.22,2212.62,1918.68,101.87,104.71,100.28
2022.03.30,1999.34,2213.20,1919.56,108.13,111.44,107.82
...,...,...,...,...,...,...
2006.04.24,NaN,NaN,NaN,64.01,NaN,NaN
2006.04.21,NaN,NaN,NaN,63.70,NaN,NaN
2006.04.20,NaN,NaN,NaN,64.40,NaN,NaN
2006.04.19,NaN,NaN,NaN,65.10,NaN,NaN
